In [19]:
from hidden import IBM_tone_analyzer_api
import pandas as pd

# IBM Watson Tone Analyzer

To read these notes directly from IBM, please visit this website: https://www.ibm.com/watson/developercloud/tone-analyzer/api/v3/python.html?python#error-handling. Otherwise, most of the notes seen bellow have been taken directly from the website above and placed here to aid the reader.

The IBM Watson Tone Analyzer service uses linguistic analysis to detect emotional and language tones in written text. The service can analyze tone at both the **document** and **sentence levels**.

## Authentication

IBM Cloud is migrating to token-based Identity and Access Management (IAM) authentication. With some service instances, you authenticate to the API by using IAM. You can pass either a bearer token in an Authorization header or an API key. If you pass in the API key, the SDK manages the lifecycle of the tokens.

In [8]:
from watson_developer_cloud import ToneAnalyzerV3

In [11]:
tone_analyzer = ToneAnalyzerV3(
    version = '2017-09-21',
    iam_apikey = IBM_tone_analyzer_api,
    url = 'https://gateway-wdc.watsonplatform.net/tone-analyzer/api'
)

## Service endpoint

The service endpoint is based on the location of the service instance. If your API endpoint URL differs from the default, you must set your endpoint. 

To find out which URL to use, view the service credentials by clicking the service instance on the Dashboard. Set the correct service ```URL``` by using the url parameter when you create the service instance or by calling the ```set_url()``` method of the service instance.

In [12]:
url = 'https://gateway-wdc.watsonplatform.net/tone-analyzer/api'

## Versioning

API requests require a version parameter that takes a date in the format ```version=YYYY-MM-DD```. When IBM changes the API in a **backwards-incompatible** way, they release a new version date.

Specify the version to use on API requests with the version parameter when you create the service instance. The service uses the API version for the date you specify, or the most recent version before that date. Don't default to the current date. Instead, specify a date that matches a version that is compatible with your app, and don't change it until your app is ready for a later version.

In [13]:
version = '2017-09-21'

## Data handling

#### Data Collection
By default, all Watson services log requests and their results. **Logging is done only to improve the services for future users. The logged data is not shared or made public.**

To prevent IBM from accessing your data for general service improvements, set the ```X-Watson-Learning-Opt-Out``` header parameter to ```true``` when you create the service instance. (Any value other than false or 0 disables request logging.) You can set the header using the ```set_default_headers``` method of the service object.

In [14]:
tone_analyzer.set_default_headers({'x-watson-learning-opt-out': "true"})

## Methods

#### Analyze general tone

Use the general purpose endpoint to analyze the tone of your input content. **The service analyzes the content for emotional and language tones**. 
- The method always analyzes the tone of the **full document; by default**, it also analyzes the **tone of each individual sentence of the content**.

You can submit no more than 128 KB of total input content and no more than 1000 individual sentences in JSON, plain text, or HTML format. The service analyzes the first 1000 sentences for document-level analysis and only the first 100 sentences for sentence-level analysis.

Per the JSON specification, the default character encoding for JSON content is effectively always UTF-8; per the HTTP specification, the default encoding for plain text and HTML is ISO-8859-1 (effectively, the ASCII character set). When specifying a content type of plain text or HTML, include the charset parameter to indicate the character encoding of the input text; for example: Content-Type: text/plain;charset=utf-8. For text/html, the service removes HTML tags and analyzes only the textual content.

#### Request

tone(self, tone_input, content_type, sentences=None, tones=None, content_language=None, accept_language=None, **kwargs)

**ToneInput**:

| Name        | Description     | 
| ------------- |:-------------:| 
| text (str):     | The input content that the service is to analyze. |



In [30]:
import json
from watson_developer_cloud import ToneAnalyzerV3

text = 'Team, I know that times are tough! Product '\
    'sales have been disappointing for the past three '\
    'quarters. We have a competitive product, but we '\
    'need to do a better job of selling it!'

tone_analysis = tone_analyzer.tone(
    {'text': text},
    'application/json').get_result()
j =(json.dumps(tone_analysis, indent=2))

# Function to Distill the Overarching Document Tones Into a Pandas DF

In [16]:
def text_analysis_to_pd(tone_analysis):
    return pd.DataFrame.from_dict(tone_analysis['document_tone']['tones'])

#### example

In [20]:
text_analysis_to_pd(tone_analysis)

,score,tone_id,tone_name
0,0.616500,sadness,Sadness
1,0.829888,analytical,Analytical


# Functions to Distill the Sentence Tones Into a Pandas DF

In [23]:
def get_score(tones):
    output = ''
    for idx, score in enumerate(tones, 1):
        output += f"Score {idx}: {score['score']} "
    return output

In [24]:
def get_tone(tones):
    output = ''
    for idx, score in enumerate(tones, 1):
        output += f"Tone {idx}: {score['tone_name']} "
    return output

In [25]:
def text_to_doc_analysis(text):
    tone_analysis = tone_analyzer.tone(
    {'text': text},
    'application/json').get_result()
    return text_analysis_to_pd(tone_analysis)

def text_to_sentence_analysis(text):
    tone_analysis = tone_analyzer.tone(
    {'text': text},
    'application/json').get_result()
    df = pd.DataFrame.from_dict(tone_analysis['sentences_tone'])
    df['score'] = df['tones'].apply(get_score)
    df['tone'] = df['tones'].apply(get_tone)
    return df[['sentence_id', 'text', 'score', 'tone']]

In [21]:
df = pd.DataFrame.from_dict(tone_analysis['sentences_tone'])

In [26]:
df['score'] = df['tones'].apply(get_score)
df['tone'] = df['tones'].apply(get_tone)

# Sample Pipeline: MBA student's "failure" essay 

#### Student's Essay

In [98]:
t = """ In my 2nd year in university, my 2 study partners and I were all working for software companies. We frequently discussed ways to make quantum career leaps. One that fascinated us was starting our own company.

One day we came up with an idea that would increase sales for consumer goods retailers and simultaneously decrease monthly consumer expenses. Each day, we polished our idea together for a couple hours.

After 2 weeks, I decided to get outside feedback. I looked for people who had at least 10 years experience in consumer goods. Finally, I convinced a friend, to connect me with a board member of the 2nd largest consumer goods retailer in my country.

I presented our business model to the board member, and he instructed his right-hand to set us meetings with managers who could evaluate our plans. Over the next month, we went to one meeting after another. The responses varied from enthusiasm to skepticism. Each time, we improved our presentation according to the feedback.

Finally, I managed to set a meeting with the previous CEO of the largest consumer goods retailer. He concluded our meeting with: “Guys, in my opinion, it’s not going to work”.

I couldn’t say if it was the pressure from school and work or the CEO’s negative feedback, but since that meeting, I wasn’t able to motivate the team to go on. We consciously gave up. 2 years later, one of my teammates called out of the blue: “check out this link…it works!”. I think he expected me to feel disappointment. Actually, I felt pride – my first business attempt was viable after all."""

#### Overarching Document Tones

In [28]:
text_to_doc_analysis(t)

,score,tone_id,tone_name
0,0.605044,joy,Joy
1,0.507810,sadness,Sadness
2,0.858529,analytical,Analytical


#### Sentence Tones

In [99]:
text_to_sentence_analysis(t)

,sentence_id,text,score,tone
0,0,"In my 2nd year in university, my 2 study partn...",Score 1: 0.559715 Score 2: 0.660207,Tone 1: Sadness Tone 2: Confident
1,1,We frequently discussed ways to make quantum c...,,
2,2,One that fascinated us was starting our own co...,,
3,3,One day we came up with an idea that would inc...,Score 1: 0.541591,Tone 1: Analytical
4,4,"Each day, we polished our idea together for a ...",Score 1: 0.684799,Tone 1: Joy
5,5,"After 2 weeks, I decided to get outside feedback.",Score 1: 0.901894 Score 2: 0.874372,Tone 1: Analytical Tone 2: Confident
6,6,I looked for people who had at least 10 years ...,Score 1: 0.508012,Tone 1: Joy
7,7,"Finally, I convinced a friend, to connect me w...",Score 1: 0.772407 Score 2: 0.579367,Tone 1: Joy Tone 2: Analytical
8,8,I presented our business model to the board me...,Score 1: 0.503542,Tone 1: Analytical
9,9,"Over the next month, we went to one meeting af...",,


In [100]:
tone_analysis = tone_analyzer.tone(
    {'text': t},
    'application/json').get_result()

In [101]:
tone_analysis['sentences_tone'][0]['tones']

[{'score': 0.559715, 'tone_id': 'sadness', 'tone_name': 'Sadness'},
 {'score': 0.660207, 'tone_id': 'confident', 'tone_name': 'Confident'}]

In [102]:
def find_sentence_tone(tone_analysis, tone):
    for d in tone_analysis['sentences_tone']:
        if len(d['tones']) > 0:
            for sent in d['tones']:
                if sent['tone_name'] == tone:
                    print(sent['score'])
                else:
                    print(f'This sentence was not {tone}')
        else:
            print(f'This sentence was not {tone}')

In [155]:
def find_sentence_tone(tone_analysis, tone):
    for d in tone_analysis['sentences_tone']:
        if len(d['tones']) > 0:
            for sent in d['tones']:
                if sent['tone_name'] == tone:
                    sentence = d['text']
                    score = sent['score']
                    level = label_score(sent['score'])
                    color = add_color(label_score(sent['score']), tone)
                    yield {
                        'sentence' : sentence,
                        'score' : score,
                        'level' : level,
                        'color' : color
                           }
                else:
                    sentence = d['text']
                    level = 'Zero'
                    yield {
                        'sentence' : sentence,
                        'score' : 0,
                        'level' : 'Zero',
                        'color' : add_color('Zero', tone)
                           }
        else:
            sentence = d['text']
            yield {
                   'sentence' : sentence,
                   'score' : 0,
                   'level' : 'Zero',
                   'color' : add_color('Zero', tone)
                    }

In [187]:
def label_score(score):
    if score < .5:
        return 'Zero'
    elif .5 <= score <= .75:
        return 'Mid'
    else:
        return 'High'

def add_color(level, tone):
    if tone == 'Analytical':
        if level == 'Zero':
            return '#21252900'
        elif level == 'Mid':
            return '#afd5fd'
        else:
            return '#6eaff1'
    if tone == 'Joy':
        if level == 'Zero':
            return '#21252900'
        elif level == 'Mid':
            return '#fffd9d'
        else:
            return '#f7f338'
    if tone == 'Sadness':
        if level == 'Zero':
            return '#21252900'
        elif level == 'Mid':
            return '#eaebec'
        else:
            return '#cacaca'
    if tone == 'Anger':
            if level == 'Zero':
                return '#21252900'
            elif level == 'Mid':
                return '#f3c9c9'
            else:
                return '#e28d8d'
    if tone == 'Tentative':
            if level == 'Zero':
                return '#21252900'
            elif level == 'Mid':
                return '#e5cdf1'
            else:
                return '#cc97e6'
    if tone == 'Confident':
            if level == 'Zero':
                return '#21252900'
            elif level == 'Mid':
                return '#bcf5c9'
            else:
                return '#44e268'
    if tone == 'Fear':
            if level == 'Zero':
                return '#21252900'
            elif level == 'Mid':
                return '#f3dee7'
            else:
                return '#f1a6c6'

In [191]:
analytical_df = pd.DataFrame(list(find_sentence_tone(tone_analysis, 'Analytical')))

In [192]:
analytical_df.to_pickle("analytical.pkl")

In [190]:
for i, d in analytical_df.iterrows():
    print(d['color'])

#21252900
#21252900
#21252900
#21252900
#21252900
#21252900
#21252900
#21252900
#21252900
#21252900
#21252900
#21252900
#21252900
#21252900
#21252900
#21252900
#21252900
#21252900
#21252900
#21252900
#21252900
#21252900
#21252900
#21252900
#21252900


In [186]:
analytical_df

,color,level,score,sentence
0,#eaebec,Mid,0.559715,"In my 2nd year in university, my 2 study partn..."
1,#21252900,Zero,0.000000,"In my 2nd year in university, my 2 study partn..."
2,#21252900,Zero,0.000000,We frequently discussed ways to make quantum c...
3,#21252900,Zero,0.000000,One that fascinated us was starting our own co...
4,#21252900,Zero,0.000000,One day we came up with an idea that would inc...
5,#21252900,Zero,0.000000,"Each day, we polished our idea together for a ..."
6,#21252900,Zero,0.000000,"After 2 weeks, I decided to get outside feedback."
7,#21252900,Zero,0.000000,"After 2 weeks, I decided to get outside feedback."
8,#21252900,Zero,0.000000,I looked for people who had at least 10 years ...
9,#21252900,Zero,0.000000,"Finally, I convinced a friend, to connect me w..."
